In [ ]:
from api import xboxAPI
import json
xuid = xboxAPI.tokens['userXUID']
print(xuid)

In [ ]:
from api import realmsAPI, getRealms
realms = getRealms()
realmId = realms[0]['id']
system_stories = realmsAPI.session.get(f'https://frontend.realms.minecraft-services.net/api/v1.0/worlds/{realmId}/stories').json()
# add `?new=true` to only get new stories
print(system_stories)
# this endpoint only returns system messages, not player posts
# however posts are returned like `userposts.xboxlive.com/users/xuid({posterXuid})/posts/{uuid}/timelines/Club/{clubId}` hinting to use the clubs api

In [ ]:
print(xboxAPI.session.get('https://' + system_stories['result'][0]['clubsPostId']).json())
# this gives some more hints about a timeline (feed) existing, plus additional specific data under the "MinecraftContent" locatorType of postContentLocators
# todo: find how to get the minecraft coordinates from the post

In [ ]:
from enum import StrEnum
class URL(StrEnum):
	# all service urls from one section of the strings
	USERPOSTS = 'https://userposts.xboxlive.com'
	CLUBPROFILE = 'https://clubprofile.xboxlive.com'
	MEDIAHUB = 'https://mediahub.xboxlive.com'
	REPUTATION = 'https://reputation.xboxlive.com'
	CLUBMODERATION = 'https://clubmoderation.xboxlive.com'
	CLUBHUB = 'https://clubhub.xboxlive.com'
	CLUBPRESENCE = 'https://clubpresence.xboxlive.com'
	COMMENTS = 'https://comments.xboxlive.com'
	ACTIVITY = 'https://avty.xboxlive.com'
	CLUBACCOUNTS = 'https://clubaccounts.xboxlive.com'

	SOCIAL = 'https://social.xboxlive.com'
	SESSIONDIRECTORY = 'https://sessiondirectory.xboxlive.com'
	PROFILE = 'https://profile.xboxlive.com'
	PEOPLEHUB = 'https://peoplehub.xboxlive.com'
	USERPRESENCE = 'https://userpresence.xboxlive.com'
	SCREENSHOTSMETADATA = 'https://screenshotsmetadata.xboxlive.com'
	USERSTATS = 'https://userstats.xboxlive.com'
	PRIVACY = 'https://privacy.xboxlive.com'
	ACHIEVEMENTS = 'https://achievements.xboxlive.com'

	POCKET = 'https://pocket.realms.minecraft.net'
	REALMS_FRONTEND = 'https://frontend.realms.minecraft-services.net'
	MULTIPLAYER = 'https://multiplayer.minecraft.net'

	def __repr__(self):
		return self.value

In [ ]:
# wraps xbox api with some useful stuff for testing
def req(url, version, method='get', headers={}, debug=False, **kwargs):
	res = xboxAPI.session.request(method, url, headers={
		'x-xbl-contract-version': str(version), # postmortem: this is actually very important! if something doesn't work, try incrementing this
		'accept-language': 'en-US',
		**headers
	}, **kwargs)
	if debug:
		print(res, res.text, res.headers)
		# try: print(json.dumps(res.json(), indent='\t'))
		# except: pass
	return res

In [ ]:
clubs_data = req(f'{URL.CLUBHUB}/clubs/xuid(' + xuid + ')/decoration/detail,settings', version=5)
realm_club = next(club for club in clubs_data.json()['clubs'] if club['clubType']['localizedTitleFamilyName'] == 'Minecraft Realm') # todo: not first only
print(realm_club)
clubId = realm_club['id']

In [ ]:
# url extracted from bedrock's strings, version from old xbox mobile app (tested 2006.0624.0047)
req(f'{URL.ACTIVITY}/clubs/clubId({clubId})/Activity/Feed?titleIds=1944307183;1739947436;1909043648;1810924247;1916611344;896928775;1828326430;1835298427;2047319603;1834660005;&imgSrc=Known', version='13').json()
# try also: `?numItems=50``

In [ ]:
# req(f'{URL.USERPOSTS}/users/me/posts')

In [ ]:
req(f'{URL.COMMENTS}/summaries/batch', version=3, method='post', json={
	'rootPaths': [
		# xbox mobile app uses 'avty.xboxlive.com/users/xuid(####)/contexts/####/activity/####' format, but timeline one provided by realms api works too
		system_stories['result'][0]['clubsPostId']
	]
}).json()

some other strings found in the bedrock executable, could potentially be useful endpoints:
* `timelines/club/BulkUnreadCount`
* `timelines/club/{0}/UnreadCount`
* `/clubs/{0}/users/xuid({1})`
* `/clubs/{0}/users/xuid({1})/session?titleFamilyId={2}`
* `clubs/Ids({0})/`
* `/clubs/{0}/reporteditems`
* `clubs/{0}/reporteditems/batch/delete`
* `{0}/likes/me`
* `{0}/comments/`
* `{0}/relationships/view/me`
* `clubs/{0}/profile`
* `/users/xuid({0})/feedback`
* `screenshotsmetadata.xboxlive.com/users/xuid(` `)/scids/00000000-0000-0000-0000-000000000000/screenshots/`
* `/screenshots/create`
* `&comp=block&blockId=QmxvY2tJZDAwMDAwMDE=`

In [ ]:
# failed brute force attempt to find the right server for the string "timelines/club/BulkUnreadCount" found in minecraft's strings
# for host in URL:
# 	print('### ' + host + ' ###')
# 	req(f'{host}/timelines/club/BulkUnreadCount', version=13, debug=True)

In [ ]:
all_screenshots = req(f'{URL.SCREENSHOTSMETADATA}/users/xuid({xuid})/screenshots', version=5, debug=True).json()
# also_all_screenshots = req(f'{URL.SCREENSHOTSMETADATA}/users/xuid({xuid})/scids/{all_screenshots['screenshots'][0]['scid']}/screenshots', version=5, debug=True)

In [ ]:
req('https://avty.xboxlive.com/timelines/club/BulkUnreadCount', method='options', version=13, debug=True)

from live memory dump:
* `https://gatherings-secondary.franchise.minecraft-services.net`
	* `https://gatherings-secondary.franchise.minecraft-services.net/api/v1.0/config/public/{uuid}?lang=en-US&clientVersion=1.21.1&clientPlatform=Windows10&clientSubPlatform=Windows10`
* `https://messaging.mktpl.minecraft-services.net`
	* `https://messaging.mktpl.minecraft-services.net/api/v1.0/session/start`
* `https://store.mktpl.minecraft-services.net`
	* `https://store.mktpl.minecraft-services.net/api/v1.0/inventory/refresh`
* `https://frontend.realms.minecraft-services.net`
* `https://safety.franchise.minecraft-services.net`
* `wss://signal.franchise.minecraft-services.net`
* `https://persona-secondary.franchise.minecraft-services.net/api/v1.0/appearance/retrieve?profilenames=personaProfile1&profilenames=personaProfile2&profilenames=personaProfile3&profilenames=personaProfile4&profilenames=personaProfile5&profilenames=personaProfile`
* `https://achievements.xboxlive.com/users/xuid({xuid}})/achievements?titleId=896928775&types=Persistent&maxItems=25`
* `https://sessiondirectory.xboxlive.com/handles/query?include=relatedInfo,customProperties` (analytics)

In [ ]:
realmsAPI.session.get(f'https://pocket.realms.minecraft.net/worlds/{realmId}/stories/settings').json()